New approach: grab all votes every day. Then store the query info in a db.
```
house_members:
    member_id:
        [{ bill_id, date, roll_call, session, chamber, vote_position}]
```

In [34]:
from utils import RenderJSON, fetch_all_items

import streamlit as st
from deta import Deta
from congress import Congress
# from datetime import 

SENATE = 'senate'
HOUSE = 'house'

DETA_API_KEY = st.secrets["DETA_API_KEY"];
PROPUBLICA_API_KEY = st.secrets["PROPUBLICA_API_KEY"];

congress = Congress(PROPUBLICA_API_KEY)
deta = Deta(DETA_API_KEY)

- For every day:
    - get all votes
    - for every vote:
        - get details
        - for every position:
            - member_id
            - bill_id
            - date
            - time
            - position
            - chamber
            - rollcall_num
            - session
            - congress

In [83]:
votes_by_day = congress.votes.by_date(HOUSE, "2022-12-21")
RenderJSON(votes_by_day)

In [82]:
votes_by_day = congress.votes.by_date(SENATE, "2022-12-21")
RenderJSON(votes_by_day)

In [81]:
vote_details = congress.votes.get(
    chamber      = votes_by_day["votes"][0]['chamber'],
    rollcall_num = votes_by_day["votes"][0]['roll_call'],
    session      = votes_by_day["votes"][0]['session'],
    congress     = votes_by_day["votes"][0]['congress'],
)
RenderJSON(vote_details)

In [79]:
def process_vote_on_date(votes_by_day):
    results = {}

    # MUST MOVE STUFF BELOW
    chamber      = votes_by_day["votes"][0]['chamber']
    rollcall_num = votes_by_day["votes"][0]['roll_call']
    session      = votes_by_day["votes"][0]['session']
    congress_n     = votes_by_day["votes"][0]['congress']


    vote_details = congress.votes.get(
        chamber      = chamber,
        rollcall_num = rollcall_num,
        session      = session,
        congress     = congress_n,
    )
    if "votes" in vote_details:
        for v in 
# NOT DONE, NEEDA LOOP THROUGH MULTIPLE RESULTS



        if "vote" in vote_details["votes"]:
            date = vote_details["votes"]["vote"]["date"]
            time = vote_details["votes"]["vote"]["time"]
            if "bill" in vote_details["votes"]["vote"]:
                bill_id = None
                if "bill_id" in vote_details["votes"]["vote"]["bill"]:
                    bill_id = vote_details["votes"]["vote"]["bill"]["bill_id"]
            if "positions" in vote_details["votes"]["vote"]:
                positions = vote_details["votes"]["vote"]["positions"]
                for position in positions:
                    member_id     = position["member_id"]
                    vote_position = position["vote_position"]
                    if member_id not in results:
                        results[member_id] = []
                    results[member_id].append({
                        "bill_id": bill_id,
                        "date": date,
                        "time": time,
                        "vote_position": vote_position,
                        "chamber": chamber,
                        "rollcall_num": rollcall_num,
                        "session": session,
                        "congress": congress_n,
                    })
    return results

RenderJSON(results)

# Putting it all together

In [74]:
from datetime import datetime, timedelta

selected_day = datetime.today()
past_limit = datetime(2023, 3, 1, 0, 0, 0) # Jan 1st, 2010
delta = timedelta(days=1)

while selected_day > past_limit:
    date_str = selected_day.strftime('%Y-%m-%d')
    # print(date_str)
    house_votes_by_day = congress.votes.by_date(HOUSE, date_str)
    # TODO: the thing!
    senate_votes_by_day = congress.votes.by_date(SENATE, date_str)
    # TODO: the thing!

    selected_day -= delta

2023-03-06
2023-03-05
2023-03-04
2023-03-03
2023-03-02
2023-03-01
2023-02-28
2023-02-27
2023-02-26
2023-02-25
2023-02-24
2023-02-23
2023-02-22
2023-02-21
2023-02-20
2023-02-19
2023-02-18
2023-02-17
2023-02-16
2023-02-15
2023-02-14
2023-02-13
2023-02-12
2023-02-11
2023-02-10
2023-02-09
2023-02-08
2023-02-07
2023-02-06
2023-02-05
2023-02-04
2023-02-03
2023-02-02
2023-02-01
2023-01-31
2023-01-30
2023-01-29
2023-01-28
2023-01-27
2023-01-26
2023-01-25
2023-01-24
2023-01-23
2023-01-22
2023-01-21
2023-01-20
2023-01-19
2023-01-18
2023-01-17
2023-01-16
2023-01-15
2023-01-14
2023-01-13
2023-01-12
2023-01-11
2023-01-10
2023-01-09
2023-01-08
2023-01-07
2023-01-06
2023-01-05
2023-01-04
2023-01-03
2023-01-02
2023-01-01
2022-12-31
2022-12-30
2022-12-29
2022-12-28
2022-12-27
2022-12-26
2022-12-25
2022-12-24
2022-12-23
2022-12-22
2022-12-21
2022-12-20
2022-12-19
2022-12-18
2022-12-17
2022-12-16
2022-12-15
2022-12-14
2022-12-13
2022-12-12
2022-12-11
2022-12-10
2022-12-09
2022-12-08
2022-12-07
2022-12-06